# Deterministic Real-Business Cycle Model

In [1]:
# Load packages
using Optim
using Plots
using Statistics
using NLsolve

In [2]:
# Functions
function fun_max(mX)
    # Find maximum of a matrix along the columns
    # Input: mX of dimension m x n
    # Output vMax of dimension n
    iCols = size(mX, 2);
    vMax = zeros(iCols,1);
    for i = 1:iCols
            temp_max = findmax( mX[:,i] );
            vMax[i] = temp_max[1];
    end
    return vMax;
end

function fun_max_index(mX)
    # Find maximum of a matrix along the columns
    # and give the index
    # Input: mX of dimension m x n
    # Output vMax of dimension n
    iCols = size(mX, 2);
    vMax_index = zeros(iCols,1);
    for i = 1:iCols
            temp_max = findmax( mX[:,i] );
            vMax_index[i] = temp_max[2];
    end
    return vMax_index;
end

fun_max_index (generic function with 1 method)

In [3]:
# Structural parameters
pAlpha = 0.36;
pBeta = 0.99;
pDelta = 0.022;
pTheta = 1;
pEta = 1;
pMu_m = 1;
pGamma = 0.05;
pGamma_n = -1/1.25;
pAlpha_m = 0.40;
pRho_x = 0.07;
pMu = 0.004;
pA = 0.05;
pC0 = 0.005;
pRho = 0.95;
pSigmaeps = 0.007;
pMuz = 1;

In [5]:
# Computing the steady state values
PARAMS = [ pAlpha, pAlpha_m, pTheta, pGamma_n, pC0, pEta, pDelta, pRho_x, pMu_m, pGamma, pBeta ];
function f!(F,x,PARAMS)

    # 1  2  3  4  5    6     7
    # c, v, s, k, n, lambda, mu

    pAlpha = PARAMS[1];
    pAlpha_m = PARAMS[2];
    pTheta = PARAMS[3];
    pGamma_n = PARAMS[4];
    pC0 = PARAMS[5];
    pEta = PARAMS[6];
    pDelta = PARAMS[7];
    pRho_x = PARAMS[8];
    pMu_m = PARAMS[9];
    pGamma = PARAMS[10];
    pBeta = PARAMS[11];

    F[1] = x[4]^pAlpha*x[5]^(1-pAlpha) -pDelta*x[4] -pC0*x[3]^pEta*(1-x[5])-pGamma*x[2]-x[1]
    F[2] = (1-pRho_x)*x[5] + pMu_m*x[2]^(1-pAlpha_m)*( x[3]*(1-x[5]) )^pAlpha_m - x[5]
    F[3] = pBeta*( pAlpha*x[4]^(pAlpha-1)*x[5]^(1-pAlpha) + (1-pDelta) ) - 1
    F[4] = pBeta*( -pTheta*x[5]^(-pGamma_n) + x[6]*( (1-pAlpha)*x[4]^pAlpha*x[5]^(-pAlpha)+pC0*x[3]^pEta ) 
                + x[7]*( (1-pRho_x)-pMu_m*pAlpha_m*x[2]^(1-pAlpha_m)*x[3]^pAlpha_m*(1-x[5])^(pAlpha_m-1) ) ) - x[7]
    F[5] = 1/x[1] - x[6]
    F[6] = x[7]*( pMu_m*(1-pAlpha_m)*x[2]^(-pAlpha_m)*( x[3]*(1-x[5]) )^(pAlpha_m) )-pGamma*x[6]
    F[7] = x[7]*pMu_m*x[2]^(1-pAlpha_m)*pAlpha_m*x[3]^(pAlpha_m-1)*(1-x[5])^pAlpha_m-x[6]*pC0*pEta*x[3]^(pEta-1)*(1-x[5])

end


SteadyState = nlsolve((F,x) ->f!(F,x,PARAMS), [ 2.67, 0.02, 2.3, 39, 0.92, 0.37, 0.01 ], iterations = 1000 , method = :newton)

# Defining the steady state
dSss = SteadyState.zero[3];
dKss = SteadyState.zero[4];
dNss = SteadyState.zero[5];


In [7]:
# Algorithm related parameters
iIter = 5000;
iGridPointsK = 30;
iGridPointsN = 2;
iToler = 10e-6;

# Constructing the grid
dKmin = dKss * 0.7;
dKmax = dKss * 1.37;
dNmin = dNss * 0.99;
dNmax = dNss * 1.01;


iStepK = (dKmax-dKmin) / (iGridPointsK-1);
vK = collect(dKmin:iStepK:dKmax);
iGridPointsK = size(vK,1);

iStepN = (dNmax-dNmin) / (iGridPointsN-1);
vN = collect(dNmin:iStepN:dNmax);
iGridPointsN = size(vN,1);

In [10]:
mK_S = zeros(iGridPointsK, iGridPointsK);
mN_S = zeros(iGridPointsN, iGridPointsN);

# Storage space
for idxKP = 1:iGridPointsK
    for idxK = 1:iGridPointsK
        mK_S[idxKP,idxK] = vK[idxK];
    end
end

for idxNP = 1:iGridPointsN
    for idxN = 1:iGridPointsN
        mN_S[idxNP,idxN] = vN[idxN];
    end
end


mK = kron( ones(iGridPointsN,iGridPointsN), mK_S  );
mKP = mK';

mN = kron( mN_S, ones(iGridPointsK,iGridPointsK) );
mNP = mN';

TEMP_A = ( 1 ./ pMu_m*dSss );
TEMP_B = ( mNP-(1-pRho_x) .* mN );
TEMP_C = ( ones(size(mN))-mN );
TEMP_D = (-pAlpha_m);
TEMP_E = ( 1 ./ (1-pAlpha_m) );

mV = ( TEMP_A .* TEMP_B .* TEMP_C .^ TEMP_D ) .^ TEMP_E;

TEMP_F = mK.^pAlpha .* mN.^(1-pAlpha);
TEMP_G = -mKP + (1-pDelta).*mK;
TEMP_H = -pC0*dSss^pEta;
TEMP_I = -pGamma.*mV;

mC = TEMP_F + TEMP_G .+ TEMP_H + TEMP_I;

#display( mC )

In [11]:
# Eliminate negative consumption
mCP = mC .> 0;
mPC = ones(iGridPointsK*iGridPointsN, iGridPointsK*iGridPointsN) - mCP;
z = 10e-5;
mC = mC.*mCP + z*mPC;

# Initialize value function
mU = log.( mC ) .+ pTheta.*( ( mN.^(1-pGamma_n) )./(1-pGamma_n) );

mCF = mC;

mVNew = log.(mCF);
global vVOld = fun_max( mU+pBeta*mVNew ); # ( [iGridPointsK*iGridPointsN] x 1 )


vEps = zeros(iIter,1);

In [12]:
global counter = 0;
### Main Iteration - BEGIN ###
for idxIter=1:iIter

 global vVOld;
 global  counter;

 vVNew = fun_max(mU+pBeta*(ones(iGridPointsN*iGridPointsK,1)*vVOld')');

 iDiff = ( abs.(vVOld-vVNew) ) ./ vVOld;

 dEps = sum(sqrt.((iDiff).^2));
 vEps[idxIter,1] = dEps;

 if sum(sqrt.((iDiff).^2)) <= iToler
  vVOld = vVNew;
  break
 else
  vVOld = vVNew;
 end
 
 counter = counter + 1;

end
### Main Iteration - END ###

print("\n");
print("\n");
print("Iterations needed to reach convergence : ")
print(counter);
print("\n");



Iterations needed to reach convergence : 1090


In [15]:
plot(vEps[100:iIter],legend=false,title="Convergence of the value iteration")
xlabel!("iterations")
ylabel!("epsilon")

# Preparing maximum values and indices for policy function
maxR = fun_max(mU+pBeta*(ones(iGridPointsK*iGridPointsN,1)*vVOld')');
idxR = fun_max_index(mU+pBeta*(ones(iGridPointsK*iGridPointsN,1)*vVOld')');

print("Maximum value");
print("\n");
print(maxR);
print("\n");
print("\n");
print("Index")
print("\n");
print(idxR);
print("\n");
print("\n");
print("Capital grid");
print("\n");
print(vK);
print("\n");
print("\n");
print("Employment grid");
print("\n");
print(vN);

Maximum value
[141.07398756940026; 141.39485913651927; 141.86114779550397; 142.3218564385664; 142.7774826690444; 143.22849229093876; 143.67532252875833; 144.11838484802686; 144.55806743573095; 144.9947373898357; 145.4162022877758; 145.81728322642618; 146.19894856424912; 146.56209700828475; 146.90756405249505; 147.23612769093018; 147.54851350145614; 147.84853098332496; 148.1445637521711; 148.438015809016; 148.7304329870146; 149.02172808992893; 149.3118182939066; 149.60062490521398; 149.88807313493615; 150.1740918891656; 150.458613573353; 150.74157390963163; 151.0229117660431; 151.30256899670096; 141.0909744134259; 141.4358846679714; 141.90058645833892; 142.35997313814957; 142.81452819416234; 143.2647046784214; 143.7109282700829; 144.15359995997812; 144.59309841362034; 145.02978205886305; 145.4513239171253; 145.85248218717038; 146.23422522597266; 146.5974517397568; 146.9429972222652; 147.2716396678581; 147.5841046551838; 147.88106988258994; 148.17553442081305; 148.46905574917426; 148.761

In [17]:
print("\n");
print("\n");

# Extending the grid where we evaluate variables
vK_L = kron( ones(iGridPointsN,1), vK );
vN_L = reshape( kron(vN, ones(iGridPointsK)), iGridPointsK*iGridPointsN,1) ;


policyC = ones(iGridPointsK*iGridPointsN,1);

for idx = 1:iGridPointsK*iGridPointsN
    policyC[idx] = mC[ floor(Int, idxR[idx]) , idx ]
end

policyC

graph1=plot(vN_L,vK_L,policyC, st = [:surface, :contourf],legend=false,title="Consumption decision")
xlabel!("N")
ylabel!("K")
savefig(graph1,"../output/graph1")

graph2=plot(vN_L,vK_L,policyC, st = [:surface], camera=(49,29),title="Consumption decision")
xlabel!("N")
ylabel!("K")
savefig(graph2,"../output/graph2")

# Extending the grid where we evaluate variables
vK_L = kron( ones(iGridPointsN,1), vK );
vN_L = kron(vN, ones(iGridPointsK));

# Constructing policy functions: Storage space
vKP = zeros(iGridPointsN*iGridPointsK,1);
vC_L = zeros(iGridPointsN*iGridPointsK,1);
vNP = zeros(iGridPointsN*iGridPointsK,1);

VA = zeros(iGridPointsN*iGridPointsK,1);
VB = zeros(iGridPointsN*iGridPointsK,1);
vV = zeros(iGridPointsN*iGridPointsK,1);

display(mC)


60×60 Array{Float64,2}:
 2.47301   3.41093  4.34812   5.2846    …  27.665     28.5911   29.5169
 1.55136   2.48928  3.42647   4.36295      26.7433    27.6694   28.5953
 0.629705  1.56763  2.50482   3.4413       25.8217    26.7478   27.6736
 0.0001    0.64598  1.58316   2.51965      24.9       25.8261   26.752
 0.0001    0.0001   0.661515  1.598        23.9784    24.9045   25.8303
 0.0001    0.0001   0.0001    0.676347  …  23.0567    23.9828   24.9087
 0.0001    0.0001   0.0001    0.0001       22.1351    23.0612   23.987
 0.0001    0.0001   0.0001    0.0001       21.2134    22.1395   23.0654
 0.0001    0.0001   0.0001    0.0001       20.2918    21.2179   22.1437
 0.0001    0.0001   0.0001    0.0001       19.3701    20.2962   21.2221
 0.0001    0.0001   0.0001    0.0001    …  18.4485    19.3746   20.3004
 0.0001    0.0001   0.0001    0.0001       17.5268    18.4529   19.3788
 0.0001    0.0001   0.0001    0.0001       16.6052    17.5313   18.4571
 ⋮                                      ⋱ 

Arrays have incorrect length or dimension.
